In [2]:
#Import Dependencies
import pandas as pd
import matplotlib as plt
import scipy.stats as st

In [12]:
state_data = 'publicAssistanceData/output_data/state_data/'
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","NewHampshire","NewJersey","NewMexico","NewYork",
  "NorthCarolina","NorthDakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "RhodeIsland","SouthCarolina","SouthDakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","WestVirginia","Wisconsin","Wyoming", "DistrictofColumbia"]
dfs = []
for state in states:
    try:
        df = pd.read_csv(f'publicAssistanceData/output_data/state_data/{state}Public.csv')
    except:
        print(state)
    dfs.append(df)

RhodeIsland


       Tornado   
 
       amountOblig  
 0         9999.00  
 1      2516339.27  
 2         2526.84  
 3        13707.00  
 4         5690.40  
 ...           ...  
 6040      7127.45  
 6041     18753.69  
 6042     12848.73  
 6043     24581.29  
 6044      4556.21  
 
 [6045 rows x 5 columns],
       disasterNumber        county           declarationDate     incidentType  \
 0               1272     Statewide  1999-05-04T00:00:00.000Z          Tornado   
 1               1272      Okmulgee  1999-05-04T00:00:00.000Z          Tornado   
 2               1272     Cleveland  1999-05-04T00:00:00.000Z          Tornado   
 3               1272      Oklahoma  1999-05-04T00:00:00.000Z          Tornado   
 4               1272     Statewide  1999-05-04T00:00:00.000Z          Tornado   
 ...              ...           ...                       ...              ...   
 4758            4453       Lincoln  2019-07-12T00:00:00.000Z  Severe Storm(s)   
 4759            4438       Lincoln  2019-06

In [49]:
#Extra cleaning of FEMA data for merging
#Remove County data
alabama_data = alabama_data.drop('county', axis=1)
alabama_data.head()
#Changing date to year
dates = alabama_data.declarationDate
years = []
for date in dates:
    year = int(date[0:4])
    years.append(year)
alabama_data['declarationDate'] = years
alabama_data = alabama_data.rename(columns={'declarationDate':'year','disasterNumber':'disasterID', 'amountOblig':'FEMA$'})
#Remove data before '03
alabama_data = alabama_data[alabama_data['year'] >= 2003]
alabama_data = alabama_data[alabama_data['year'] <= 2018]
alabama_data.head()

,disasterID,year,incidentType,FEMA$
348,1466,2003,Severe Storm(s),93998.29
349,1466,2003,Severe Storm(s),13497.98
350,1466,2003,Severe Storm(s),49107.89
351,1466,2003,Severe Storm(s),395072.54
352,1466,2003,Severe Storm(s),3750.00


In [70]:
#Group the data to by year and disaster
alabama_group = alabama_data.groupby(['year', 'disasterID'])
disaster_types = alabama_group['incidentType'].unique()
disaster_types_fix = [dis[0] for dis in disaster_types]
alabama_group_df = pd.DataFrame({'FEMA$':alabama_group['FEMA$'].sum(), 'incidentType':disaster_types_fix})
alabama_group_df.head()

FEMA$     incidentType
year disasterID                               
2003 1466        1.213776e+07  Severe Storm(s)
2004 1549        2.182246e+08        Hurricane
2005 1593        1.986408e+07        Hurricane
     1605        1.168102e+08        Hurricane
     3237        2.942631e+06        Hurricane

In [24]:
#Read in DD summary
declare_data = pd.read_csv('disaster declaration sum2.csv')
declare_data.head()

,Unnamed: 0,disasterNumber,fipsStateCode,fyDeclared,declarationType,incidentType,fipsCountyCode,state county
0,0,4,26,1953,DR,Tornado,0,"26,0"
1,1,2,48,1953,DR,Tornado,0,"48,0"
2,2,9,48,1953,DR,Flood,0,"48,0"
3,3,11,33,1953,DR,Fire,0,"33,0"
4,4,6,26,1953,DR,Tornado,0,"26,0"


In [25]:
#Clean decleration data
declare_data = declare_data.drop(['Unnamed: 0', 'fipsCountyCode', 'state county', 'fyDeclared', 'incidentType', 'fipsStateCode'], axis=1)
declare_data.head()

,disasterNumber,declarationType
0,4,DR
1,2,DR
2,9,DR
3,11,DR
4,6,DR
